<a href="https://colab.research.google.com/github/rshahdelhi/PetProject/blob/master/feature_engineering_2_pca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

In [ ]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

useducation_df = pd.read_sql_query('select * from useducation',con=engine)

# No need for an open connection, 
# because you're only doing a single query
engine.dispose()


In [ ]:
from os import EX_DATAERR
fill_list = [ 'ENROLL', 'TOTAL_REVENUE',
       'FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE',
       'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE',
       'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE',
       'CAPITAL_OUTLAY_EXPENDITURE', 'GRADES_PK_G', 'GRADES_KG_G',
       'GRADES_4_G', 'GRADES_8_G', 'GRADES_12_G', 'GRADES_1_8_G',
       'GRADES_9_12_G', 'GRADES_ALL_G']

states = useducation_df["STATE"].unique()

for state in states:
  useducation_df.loc[useducation_df["STATE"] == state, fill_list] = useducation_df.loc[useducation_df["STATE"] == state, fill_list].interpolate()

useducation_df.dropna(inplace=True)   

In [ ]:
diff = useducation_df["GRADES_4_G"].sum() - useducation_df["GRADES_8_G"].sum()
print(diff)


-186982.5


In [ ]:
useducation_df["overall_score"] = ((useducation_df["GRADES_4_G"] *  ( useducation_df['AVG_MATH_4_SCORE'] + useducation_df['AVG_READING_4_SCORE']) * 0.5 ) + (useducation_df["GRADES_8_G"] *  ( useducation_df['AVG_MATH_8_SCORE'] + useducation_df['AVG_READING_8_SCORE']) * 0.5 )  )/ useducation_df["GRADES_4_G"] + useducation_df["GRADES_8_G"]


In [ ]:
useducation_df[['overall_score','TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE',
       'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE',
       'CAPITAL_OUTLAY_EXPENDITURE']].corr()

,overall_score,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
overall_score,1.000000,0.925197,0.882191,0.929027,0.965937,0.935366
TOTAL_EXPENDITURE,0.925197,1.000000,0.990255,0.992988,0.943702,0.925784
INSTRUCTION_EXPENDITURE,0.882191,0.990255,1.000000,0.974399,0.902173,0.884798
SUPPORT_SERVICES_EXPENDITURE,0.929027,0.992988,0.974399,1.000000,0.952282,0.911245
OTHER_EXPENDITURE,0.965937,0.943702,0.902173,0.952282,1.000000,0.918188
CAPITAL_OUTLAY_EXPENDITURE,0.935366,0.925784,0.884798,0.911245,0.918188,1.000000


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = useducation_df[['INSTRUCTION_EXPENDITURE',
       'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE',       'CAPITAL_OUTLAY_EXPENDITURE']]

X = StandardScaler().fit_transform(X)

sklearn_pca = PCA(n_components=1)
useducation_df["pca_1"] = sklearn_pca.fit_transform(X)

print(
    'The percentage of total variance in dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)


The percentage of total variance in dataset explained by each component from Sklearn PCA.
 [0.9430175]


In [ ]:
useducation_df[['overall_score',"pca_1",'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE',
       'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE',
       'CAPITAL_OUTLAY_EXPENDITURE']].corr()

,overall_score,pca_1,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
overall_score,1.000000,0.955700,0.925197,0.882191,0.929027,0.965937,0.935366
pca_1,0.955700,1.000000,0.992054,0.968606,0.988345,0.971343,0.955792
TOTAL_EXPENDITURE,0.925197,0.992054,1.000000,0.990255,0.992988,0.943702,0.925784
INSTRUCTION_EXPENDITURE,0.882191,0.968606,0.990255,1.000000,0.974399,0.902173,0.884798
SUPPORT_SERVICES_EXPENDITURE,0.929027,0.988345,0.992988,0.974399,1.000000,0.952282,0.911245
OTHER_EXPENDITURE,0.965937,0.971343,0.943702,0.902173,0.952282,1.000000,0.918188
CAPITAL_OUTLAY_EXPENDITURE,0.935366,0.955792,0.925784,0.884798,0.911245,0.918188,1.000000
